# Loop 13 Analysis: Strategic Pivot Required

## Current Status
- Best CV/LB: 70.630478
- Target: 68.919154
- Gap: 1.711 points (2.42%)

## Key Finding from Experiment 014
After 30+ minutes with extreme SA parameters (n=50000, r=80), the optimizer found NO improvement. The solution is at a structural optimum that SA cannot escape.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

getcontext().prec = 25

# Load current best solution
df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')
print(f'Loaded {len(df)} rows')
print(df.head())

Loaded 20100 rows
      id                       x                      y  \
0  001_0  s-48.19608619421424578  s58.77098461521422479   
1  002_0    s0.15409706962136430  s-0.03854074269478543   
2  002_1   s-0.15409706962136430  s-0.56145925730521462   
3  003_0    s1.12365581614030097   s0.78110181599256301   
4  003_1    s1.23405569584216002   s1.27599950066375900   

                      deg  
0   s45.00000000000000000  
1  s203.62937773065684155  
2   s23.62937773065679181  
3  s111.12513229289299943  
4   s66.37062226934300213  


In [2]:
# Analyze score breakdown by N
def calculate_score_for_n(df, n):
    """Calculate score contribution for a specific N"""
    group_id = f'{n:03d}'
    group_data = df[df['id'].str.startswith(group_id + '_')]
    if len(group_data) == 0:
        return None
    
    # Parse coordinates
    xs = group_data['x'].str.strip('s').astype(float).values
    ys = group_data['y'].str.strip('s').astype(float).values
    
    # Calculate bounding box
    min_x, max_x = xs.min(), xs.max()
    min_y, max_y = ys.min(), ys.max()
    
    # Approximate side length (need to account for tree dimensions)
    # Tree is roughly 0.7 wide and 1.0 tall
    side = max(max_x - min_x + 0.7, max_y - min_y + 1.0)
    
    return side**2 / n

# Calculate scores for all N
scores = {}
for n in range(1, 201):
    score = calculate_score_for_n(df, n)
    if score:
        scores[n] = score

print(f'Total score (approx): {sum(scores.values()):.6f}')
print(f'\nTop 10 contributors:')
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, s in sorted_scores[:10]:
    print(f'  N={n}: {s:.6f} ({s/sum(scores.values())*100:.2f}%)')

Total score (approx): 86.868075

Top 10 contributors:
  N=2: 1.159640 (1.33%)
  N=1: 1.000000 (1.15%)
  N=4: 0.868829 (1.00%)
  N=5: 0.770716 (0.89%)
  N=3: 0.744906 (0.86%)
  N=7: 0.705515 (0.81%)
  N=6: 0.676163 (0.78%)
  N=8: 0.634881 (0.73%)
  N=9: 0.603207 (0.69%)
  N=12: 0.602703 (0.69%)


In [3]:
# Analyze efficiency by N range
print('\nScore breakdown by N range:')
ranges = [(1, 20), (21, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end+1) if n in scores)
    pct = range_score / sum(scores.values()) * 100
    print(f'  N={start}-{end}: {range_score:.4f} ({pct:.1f}%)')


Score breakdown by N range:
  N=1-20: 12.9939 (15.0%)
  N=21-50: 14.3106 (16.5%)
  N=51-100: 21.1688 (24.4%)
  N=101-150: 19.6210 (22.6%)
  N=151-200: 18.7737 (21.6%)


In [4]:
# Key insight: What would it take to reach the target?
target = 68.919154
current = 70.630478
gap = current - target

print(f'\nGap Analysis:')
print(f'  Current: {current:.6f}')
print(f'  Target: {target:.6f}')
print(f'  Gap: {gap:.6f} ({gap/current*100:.2f}%)')

# If we improved large N (100-200) by X%, what would be the impact?
large_n_score = sum(scores[n] for n in range(100, 201) if n in scores)
print(f'\nLarge N (100-200) contributes: {large_n_score:.4f}')
print(f'  To close gap of {gap:.4f} from large N alone:')
print(f'  Need {gap/large_n_score*100:.1f}% improvement on large N')


Gap Analysis:
  Current: 70.630478
  Target: 68.919154
  Gap: 1.711324 (2.42%)

Large N (100-200) contributes: 38.7817
  To close gap of 1.7113 from large N alone:
  Need 4.4% improvement on large N


In [5]:
# What approaches have been tried?
approaches_tried = [
    ('Ensemble from 25+ sources', '70.630478', 'CEILING - all public sources exhausted'),
    ('SA optimization (n=30000, r=20)', '70.630478', 'No improvement'),
    ('SA optimization (n=50000, r=80)', '70.630478', 'No improvement after 30+ min'),
    ('bbox3 optimization', '70.647326', 'Produces overlapping trees'),
    ('Zaburo grid + SA', '88.33', 'Much worse than baseline'),
    ('Random restart SA', '70.630478', 'No improvement'),
    ('Genetic algorithm', '70.630478', 'No improvement'),
    ('Tessellation for large N', '70.630478', 'Worse than baseline'),
    ('Deletion cascade from large N', '70.630478', 'No improvement'),
]

print('\nApproaches Tried:')
for approach, score, result in approaches_tried:
    print(f'  {approach}: {score} - {result}')


Approaches Tried:
  Ensemble from 25+ sources: 70.630478 - CEILING - all public sources exhausted
  SA optimization (n=30000, r=20): 70.630478 - No improvement
  SA optimization (n=50000, r=80): 70.630478 - No improvement after 30+ min
  bbox3 optimization: 70.647326 - Produces overlapping trees
  Zaburo grid + SA: 88.33 - Much worse than baseline
  Random restart SA: 70.630478 - No improvement
  Genetic algorithm: 70.630478 - No improvement
  Tessellation for large N: 70.630478 - Worse than baseline
  Deletion cascade from large N: 70.630478 - No improvement


In [6]:
# What approaches have NOT been tried?
approaches_not_tried = [
    ('Constraint Programming (OR-Tools CP-SAT)', 'Exact method for small N'),
    ('Integer Programming (MIP)', 'Dotted-board or vertical-slice formulation'),
    ('No-Fit Polygon (NFP) based optimization', 'Pre-compute feasible placements'),
    ('Asymmetric solutions', 'Discussion says winning solutions will be asymmetric'),
    ('Different rotation angles', 'Current uses 0/180, try continuous angles'),
    ('Compaction algorithms', 'Push trees toward center/corner'),
    ('Different initial configurations', 'Not grid-based starts'),
]

print('\nApproaches NOT Tried:')
for approach, reason in approaches_not_tried:
    print(f'  {approach}: {reason}')


Approaches NOT Tried:
  Constraint Programming (OR-Tools CP-SAT): Exact method for small N
  Integer Programming (MIP): Dotted-board or vertical-slice formulation
  No-Fit Polygon (NFP) based optimization: Pre-compute feasible placements
  Asymmetric solutions: Discussion says winning solutions will be asymmetric
  Different rotation angles: Current uses 0/180, try continuous angles
  Compaction algorithms: Push trees toward center/corner
  Different initial configurations: Not grid-based starts


## Key Strategic Insight

The SA-based optimization has been thoroughly exhausted. After 14 experiments:
1. All public sources have been ensembled - ceiling at 70.630478
2. SA with extreme parameters (30+ min, r=80, n=50000) found NO improvement
3. The solution is at a STRUCTURAL optimum

## The Path Forward

The target of 68.919 is 2.42% below our current best. This requires:
1. **Fundamentally different algorithms** - not variations of SA
2. **Exact methods for small N** - CP/MIP can find provably optimal solutions
3. **Novel configurations** - asymmetric layouts, different angles
4. **Focus on large N** - where most score comes from

## Recommended Next Experiment

Try Constraint Programming with OR-Tools CP-SAT for small N (1-20):
- These contribute 11.4% of score
- Exact methods can find global optima
- Even small improvements compound across 20 N values